In [1]:
# Importation

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests 
import pandas as pd
import numpy as np

# Scrapping jeuxvideos.com

## Fonction get_data(link)

In [ ]:
def get_data(link):

  """
  Fonction qui prend en argument un url du site jeuxvideos.com, rubrique 'tous les jeux'
  et qui renvoit un DataFrame contenant le titre du jeu, les notes de tests et les notes d'avis.
  """
  """
  La fonction différencie la première page des autres car les éléments de scrapping diffèrent sur la première page.
  """

  page = requests.get(link)
  page.text
  soup = bs(page.text, "html.parser")

  titles = []
  test = []
  avis = []

  # Si le lien est celui de la premiere page du site
  if link == 'https://www.jeuxvideo.com/tous-les-jeux/':

    # Titres
    for element in soup.select("h2 > a"):
      titles.append(element.text)

    # Notes test
    for element in soup.find_all('div', class_ = 'gameHub__2Gf7Q4'):
      if element.find_next('span', class_ = 'editorialRating__1tYu_r'):
        test.append(element.find_next('span', class_ = 'editorialRating__1tYu_r').text.strip())
      else :
        test.append('Non renseigné')

    # Notes avis
    for element in soup.find_all('div', class_ = 'gameHub__2Gf7Q4'):
      if element.find_next('span', class_ = 'userRating__1y96su'):
        avis.append(element.find_next('span', class_ = 'userRating__1y96su').text.strip())
      else :
        avis.append('Non renseigné')

    # Création d'un DataFrame
    df = pd.DataFrame({'titres': titles,'test': test, 'avis':avis})
  
  # Sinon le lien correpond à une page ultérieure du type https://www.jeuxvideo.com/tous-les-jeux/?=NUMERO_DE_PAGE

  else :
    
    # Titres
    name = soup.find_all('h2', attrs = {'class': 'gameTitle__3PBZI1'})
    
    for element in name :
      titles.append(element.find('span', attrs = {'class' : 'gameTitleLink__196nPy'}).text)

    # Récupération des notes
    notes = soup.find_all('div', attrs = {'class' : 'gameHub__2Gf7Q4'})

    for element in notes :

      # Notes test
      try :
        if element.find('span', attrs = {'class' : 'editorialRating__1tYu_r'}):
          test.append(element.find('span', attrs = {'class' : 'editorialRating__1tYu_r'}).text)
        elif element.find('span', attrs = {'class' : 'editorialRating__1tYu_r'}) == None : #si le None n'est pas renseigné, l'except n'est pas lu et la liste est raccourcie d'un élément
          test.append('Non renseigné')
      except :
          test.append('Non renseigné')

      # Notes avis
      try :
        if element.find('span', attrs = {'class' : 'userRating__1y96su'}):
          avis.append(element.find('span', attrs = {'class' : 'userRating__1y96su'}).text)
        elif element.find('span', attrs = {'class' : 'userRating__1y96su'}) == None :
          avis.append('Non renseigné')
      except :
          avis.append('Non renseigné')
          
    # Création d'un DataFrame 
    df = pd.DataFrame({'titres' : titles,'test':test,'avis':avis})

  return df

## Fonction get_data_jeu(link)

In [ ]:
def get_data_jeu(link):

  """fonction qui prend en argument une page du site jeuxvideos.com référençant un jeu
  et qui renvoit une liste d'information sur ce jeu. Les informations sont :

  - Nom du jeu
  - Note d'avis
  - Note de test
  - Platformes disponibles
  
  Format de l'URL :
  'https://www.jeuxvideo.com/jeux/jeu-1716149/' """

  # Initialisation
  page = requests.get(link)
  page.text
  soup = bs(page.text, "lxml")

  # Instanciation de listes vides
  liste_plateformes = []
  liste_publisher = []
  donnees_jeuxvideos = []

  # Nom du jeu vidéo
  name = soup.find('h1', attrs = {'class' : 'breadcrumb__item'}).text

  # Notes du jeu : instanciation d'un bloc Try pour lever les exceptions (tous les jeus ne sont pas notés)
  try :
    if soup.find_all('text', attrs = {'class' : 'gameCharacteristicsMain__gaugeText'}): # si la partie note existe sur la page
      test = soup.find_all('text', attrs = {'class' : 'gameCharacteristicsMain__gaugeText'})[0].text # on récupère les 2 premiers élements text
      avis = soup.find_all('text', attrs = {'class' : 'gameCharacteristicsMain__gaugeText'})[1].text
  except : # si la partie note n'existe pas, on lève l'exception en renseignant "Non trouvé"
      test = 'Non trouvé'
      avis = 'Non trouvé'

  # Plateforme de jeu
  support = soup.find_all('li', attrs = {'class' : 'gameHeaderBanner__platform'})
  
  for element in support :
    liste_plateformes.append(element.text.strip()) # certaines données contiennent des espaces et des sauts de lignes

  # Publisher
  publisher = soup.find('div', attrs = {'class':"gameCharacteristicsDetailed__tr"})
  span = publisher.find_all('span')

  for element in span :
    liste_publisher.append(element.text.strip())


  donnees_jeuxvideos.append(name)
  donnees_jeuxvideos.append(test)
  donnees_jeuxvideos.append(avis)
  donnees_jeuxvideos.append(liste_plateformes)
  donnees_jeuxvideos.append(liste_publisher)
  
  return donnees_jeuxvideos

## Fonction get_data_n_pages(n)

In [ ]:
def get_data_n_pages(n_pages):

  """ Fonction qui prend en argument un nombre de page sur le site jeuxvideos.com
  et qui retourne un DataFrame contenant les informations sur les jeux videos.
  """
  # Instanciation d'un DataFrame vide
  data_jeuxvideos = pd.DataFrame()

  # DataFrame correspondant à la 1ère page
  df1 = get_data('https://www.jeuxvideo.com/tous-les-jeux/')

  # Concaténation du DF de la 1ère page avec le DF vide
  data_jeuxvideos = pd.concat([data_jeuxvideos, df1])

  # Récupération des DF des autres pages et concaténation
  for i in range(2,n_pages,1) :

    link = 'https://www.jeuxvideo.com/tous-les-jeux/?p=' + str(i)
    data_jeuxvideos = pd.concat([data_jeuxvideos, get_data(link)])

  # La fonction renvoit le DataFrame final
  return data_jeuxvideos.reset_index().drop('index', axis = 1)


In [2]:
# Le scrapping du site a eu lieu le 19 avril et le dataframe généré a été enregistré sous format
# CSV sous le nom dataframe_jeuxvideos.csv
# La mise à jour régulière du site jeuxvideos.com peut générer des erreurs

path = "dataframe_jeuxvideos.csv"

data = pd.read_csv(path)
data.head()

,Unnamed: 0,titres,test,avis
0,0,Hogwarts Legacy : l'Héritage de Poudlard,18/20,15.5/20
1,1,Resident Evil 4 (2023),18/20,17.3/20
2,2,Diablo IV,- -/20,- -/20
3,3,The Legend of Zelda : Tears of the Kingdom sur...,- -/20,- -/20
4,4,The Last of Us Part I,17/20,16.2/20


In [3]:
data = data.drop('Unnamed: 0', axis = 1)
data.head()

,titres,test,avis
0,Hogwarts Legacy : l'Héritage de Poudlard,18/20,15.5/20
1,Resident Evil 4 (2023),18/20,17.3/20
2,Diablo IV,- -/20,- -/20
3,The Legend of Zelda : Tears of the Kingdom sur...,- -/20,- -/20
4,The Last of Us Part I,17/20,16.2/20


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43940 entries, 0 to 43939
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titres  43940 non-null  object
 1   test    43940 non-null  object
 2   avis    43940 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB


# Scrapping metacritic.com

In [ ]:
#import des modules de scrapping

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit
import requests
import pandas as pd
import numpy as np


In [ ]:
#choisir l'url de metacritic (page "Best video games of all time")
url = 'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered'

#méthode trouvée sur stackoverflow pour résoudre le problème 'requests.exceptions.TooManyRedirects: Exceeded 30 redirects'
page = requests.Session()
page.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
réponse = page.get(url)

#Contenu HTML et réponse: afficher le contenu du site web.
soup = bs(réponse.content,'html.parser')

In [ ]:
#Création des listes pour le dataframe

Name = []
Date = []
Platform = []
Note = []
P_Note = []

# Extraction des éléments de la page qui contiennent les données recherchées
game_elements = soup.findAll("td", class_="clamp-summary-wrap")

# Parcourir les éléments pour extraire les données

for game_element in game_elements:
    # Extraire le nom du jeu
    game = game_element.find("a", class_='title').text.strip()
    Name.append(game)
    
    # Extraire la note
    score = game_element.find("div", class_="metascore_w large game positive").text.strip()
    Note.append(score)

    # Extraire la note des joueurs, le .text pose ici problème pour afficher les résultats.
    # On note que le résulat s'affiche entre crochets, il faudra les supprimer par la suite.

    p_score = game_element.find("div", class_="metascore_w user large game positive")
    P_Note.append(p_score)
    
    # Extraire la plateforme

    platform = game_element.find("div", class_="platform").find("span", class_="data").text.strip()
    Platform.append(platform)
    
    # Extraire la date, ici obligation d'utiliser -1 pour séléctionner le dernier span qui nous intéresse, en l'occurence la date

    date = game_element.find("div", class_="clamp-details").find_all('span')[-1].text.strip()
    Date.append(date)

# Créer un DataFrame avec les données extraites
df_metacritic = pd.DataFrame({
    "Name": Name,
    "Platform": Platform,
    "Date": Date,
    "Note MC": Note,
    "Note Players MC": P_Note,
})

#export en csv

#df_metacritic.to_csv("metacritic_data.csv", index=False)

In [ ]:
df_metacritic.head(50)

,Name,Platform,Date,Note MC,Note Players MC
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,[9.1]
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,[7.5]
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,[7.8]
3,SoulCalibur,Dreamcast,"September 8, 1999",98,[8.5]
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,[8.0]
5,Super Mario Galaxy,Wii,"November 12, 2007",97,[9.1]
6,Super Mario Galaxy 2,Wii,"May 23, 2010",97,[9.1]
7,Red Dead Redemption 2,Xbox One,"October 26, 2018",97,[8.3]
8,Grand Theft Auto V,Xbox One,"November 18, 2014",97,[7.9]
9,Grand Theft Auto V,PlayStation 3,"September 17, 2013",97,[8.3]


In [ ]:
df_metacritic['Note MC'] = df_metacritic['Note MC'].astype(float)

In [ ]:
print(df_metacritic[df_metacritic['Note Players MC'].isna()].index)

Int64Index([12, 22, 51, 62, 75, 80], dtype='int64')


In [ ]:
Name = []
Note = []
P_Note = []
Platform = []
Date = []
page = 0

while page != 80:
    url = f"https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={page}"
    session = requests.Session()
    session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    response = session.get(url)

    soup = bs(response.content,'html.parser')
    game_elements = soup.findAll("td", class_="clamp-summary-wrap")

    for game_element in game_elements:
        
        game = game_element.find("a", class_='title').text.strip()
        Name.append(game)

        score = game_element.find("div", class_="metascore_w large game positive").text.strip()
        Note.append(score)

        p_score = game_element.find("div", class_="metascore_w user large game positive")
        P_Note.append(p_score)

        platform = game_element.find("div", class_="platform").find("span", class_="data").text.strip()
        Platform.append(platform)

        date = game_element.find("div", class_="clamp-details").find_all('span')[-1].text.strip()
        Date.append(date)

    page += 1
        

# Créer un DataFrame avec les données extraites
df_metacritic = pd.DataFrame({
    "Name": Name,
    "Platform": Platform,
    "Date": Date,
    "Note MC": Note,
    "Note Players MC": P_Note,
})

df_metacritic.head(50)

,Name,Platform,Date,Note MC,Note Players MC
0,Ninja Gaiden Black,Xbox,"September 20, 2005",94,[8.7]
1,The Elder Scrolls IV: Oblivion,PC,"March 20, 2006",94,[8.3]
2,Elden Ring,PC,"February 25, 2022",94,None
3,Street Fighter IV,PlayStation 3,"February 17, 2009",94,None
4,Super Mario Advance 4: Super Mario Bros. 3,Game Boy Advance,"October 21, 2003",94,[8.9]
5,Jet Grind Radio,Dreamcast,"October 30, 2000",94,[8.3]
6,Metal Gear Solid,PlayStation,"October 21, 1998",94,[8.4]
7,Grim Fandango,PC,"October 14, 1998",94,[9.0]
8,Tom Clancy's Splinter Cell Chaos Theory,Xbox,"March 28, 2005",94,[9.0]
9,Resident Evil Code: Veronica,Dreamcast,"February 29, 2000",94,[9.5]


In [ ]:
df_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7900 entries, 0 to 7899
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             7900 non-null   object
 1   Platform         7900 non-null   object
 2   Date             7900 non-null   object
 3   Note MC          7900 non-null   object
 4   Note Players MC  4999 non-null   object
dtypes: object(5)
memory usage: 308.7+ KB


In [ ]:
df_metacritic['Platform'].value_counts()

PC                  2271
PlayStation 4        798
Switch               760
Xbox 360             606
Xbox One             523
PlayStation 3        503
PlayStation 2        461
Xbox                 301
DS                   187
PlayStation 5        182
Wii                  174
GameCube             170
PSP                  152
Game Boy Advance     151
3DS                  140
Xbox Series X        128
PlayStation Vita      99
Wii U                 96
PlayStation           91
Dreamcast             61
Nintendo 64           44
Stadia                 2
Name: Platform, dtype: int64

In [ ]:
df_metacritic['Platform'].nunique()

22

In [ ]:
#Création des listes

Name = []
Note = []
P_Note = []
Platform = []
Date = []
page = 0

#Liste while pour récupérer les 90 premières pages avec la boucle while + for + try en cas d'erreur

while page != 203:
    url = f"https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={page}"
    session = requests.Session()
    session.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
    response = session.get(url)

    soup = bs(response.content,'html.parser')
    game_elements = soup.findAll("td", class_="clamp-summary-wrap")

    for game_element in game_elements:
        try:
            game = game_element.find("a", class_='title')
            if game is not None:
                game = game.text.strip()
            Name.append(game)

            score = game_element.find("div", class_="metascore_w large game positive") or game_element.find("div", class_="metascore_w large game mixed") or game_element.find("div", class_="metascore_w medium game negative")
            if score is not None:
                score = score.text.strip()
            Note.append(score)

            p_score = game_element.find("div", class_="metascore_w user large game positive") or game_element.find("div", class_="metascore_w user large game mixed") or game_element.find("div", class_="metascore_w user medium game negative")
            if p_score is not None:
                p_score = p_score.text.strip()
            P_Note.append(p_score)

            platform = game_element.find("div", class_="platform").find("span", class_="data")
            if platform is not None:
                platform = platform.text.strip()
            Platform.append(platform)

            date = game_element.find("div", class_="clamp-details").find_all('span')[-1]
            if date is not None:
                date = date.text.strip()
            Date.append(date)

        except Exception as e:
            print(f"Erreur détectée sur la page {page}: {e}")

    page += 1
        
# Créer un DataFrame avec les données extraites
df_metacritic = pd.DataFrame({
    "Name": Name,
    "Platform": Platform,
    "Date": Date,
    "Note MC": Note,
    "Note Players MC": P_Note,
})

df_metacritic.head()

,Name,Platform,Date,Note MC,Note Players MC
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98,7.5
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98,7.8
3,SoulCalibur,Dreamcast,"September 8, 1999",98,8.5
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98,8.0


Seulement 3 minutes 30 pour scraper l'ensemble des pages les erreurs ralentissaient le bon déroulement du code. Sur 20205 entrées:

- 93% de réussite pour les Notes MC
- 84% de réussite pour les Notes Players MC


In [ ]:
df_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20205 entries, 0 to 20204
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             20205 non-null  object
 1   Platform         20205 non-null  object
 2   Date             20205 non-null  object
 3   Note MC          18985 non-null  object
 4   Note Players MC  17150 non-null  object
dtypes: object(5)
memory usage: 789.4+ KB


In [ ]:
df_metacritic.to_csv("MC_final.csv", index=False)

In [ ]:
df = pd.read_csv('MC_final.csv')
df.head(50)


,Name,Platform,Date,Note MC,Note Players MC
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998",99.0,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",98.0,7.5
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",98.0,7.8
3,SoulCalibur,Dreamcast,"September 8, 1999",98.0,8.5
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",98.0,8.0
5,Super Mario Galaxy,Wii,"November 12, 2007",97.0,9.1
6,Super Mario Galaxy 2,Wii,"May 23, 2010",97.0,9.1
7,Red Dead Redemption 2,Xbox One,"October 26, 2018",97.0,8.3
8,Grand Theft Auto V,Xbox One,"November 18, 2014",97.0,7.9
9,Grand Theft Auto V,PlayStation 3,"September 17, 2013",97.0,8.3


Test Selenium pour scrapper le nom des platform 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import numpy as np

Créer instance driver pour chrome

In [ ]:
try:
    driver = webdriver.Chrome()
except:
    driver = webdriver.Chrome(ChromeDriverManager().install())

accéder à la page

In [ ]:
driver.get('https://www.metacritic.com/browse/games/score/metascore/all/all/filtered')

Refuser les cookies

In [ ]:
webelement = driver.find_element(by='id', value = "onetrust-reject-all-handler")

driver.execute_script("arguments[0].click();", webelement)

Cliquer sur le premier lien

In [ ]:
liens = driver.find_elements_by_class_name("title")

for lien in liens:
    lien.click()

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_class_name'

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

publishers = []
page = 0
while page != 5:
    url = f'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={page}'
    response = requests.get(url, headers=headers)
    soup = bs(response.content, 'html.parser')
    games = soup.find_all('a', class_='title')
    
    for game in games:
        game_url = 'https://www.metacritic.com' + game['href']
        game_response = requests.get(game_url, headers=headers)
        game_soup = bs(game_response.content, 'html.parser')
        publisher = game_soup.find('li', class_='summary_detail publisher').find('span', class_='data').text.strip()
        publishers.append(publisher)
    
    page += 1

print(publishers)

['Nintendo', 'Activision', 'Rockstar Games', 'Namco', 'Rockstar Games', 'Nintendo', 'Nintendo', 'Rockstar Games', 'Rockstar Games', 'Rockstar Games', 'ZA/UM', 'Rockstar Games', 'Activision', 'Nintendo', 'Activision', 'Nintendo', 'Rockstar Games', 'Rockstar Games', 'Nintendo', 'Rockstar Games', 'Nintendo', 'Microsoft Game Studios', 'Sega', 'VU Games', 'Rockstar Games', 'Nintendo', '2K Games', 'Nintendo', 'SCEA', 'Capcom', 'EA Games', 'EA Games', 'Warner Bros. Interactive Entertainment', 'Valve Software', 'Namco', 'Bandai Namco Games', 'Electronic Arts', 'Limited Run Games', 'Nintendo', 'Bandai Namco Games', 'Bethesda Softworks', 'Sierra Entertainment', 'Capcom', 'Nintendo', 'SCEA', '2K Games', 'Konami', 'Rockstar Games', 'Interplay', 'Rockstar Games', 'Rockstar Games', 'SCEA', 'Nintendo', 'Rockstar Games', 'SCEA', 'Microsoft Game Studios', 'Sega\n                                                    \n\t\t\t\t\t, \t\t\t\t\t\t\n                                                        Atlus'

Cela fonctionne mais après réflexion il a été décidé que cette données n'était pas nécéssaire

# Scrapping vgchartz.com

On souhaite scrapper le site suivant: https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&showvgchartzscore=1&shownasales=0&showdeveloper=0&showcriticscore=0&showcriticscore=1&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showlastupdate=0&showlastupdate=1&showothersales=0&showshipped=0&showshipped=1

les informations qui nous intéressent sont les suivantes:

Nom du jeu
Plateforme
Editeur
Total Shipped
Release Date

In [ ]:
#import des modules de scrapping

from bs4 import BeautifulSoup as bs
from bs4 import UnicodeDammit
import requests
import pandas as pd
import numpy as np
from urllib.request import urlopen



In [ ]:
#choisir l'url de VGChartz
url_VG = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&showvgchartzscore=1&shownasales=0&showdeveloper=0&showcriticscore=0&showcriticscore=1&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showlastupdate=0&showlastupdate=1&showothersales=0&showshipped=0&showshipped=1'

page_VG = urlopen(url_VG)

soup = bs(page_VG, 'html.parser')


In [ ]:
#Création des listes

Name = []
publishers = []
sales = []
release_dates = []
consoles = []


td_elements = soup.find_all('td', style='font-size:12pt;')

# parcourir chaque élément <td> et extraire le texte
for td in td_elements:
    titre = td.text.strip()
    Name.append(titre)

# Extraction des informations pour chaque jeu
for game in soup.find_all('tr', {'style': ['background-image:url(../imgs/chartBar_large.gif); height:70px', 'background-image:url(../imgs/chartBar_alt_large.gif); height:70px', 'font-size:12pt;']}):
    # Extraction du nom du publisher
    publisher = game.find_all('td')[4].text.strip()
    publishers.append(publisher)
    
    # Extraction du nombre de vente
    sale = game.find_all('td')[8].text.strip()
    sales.append(sale)
    
    # Extraction de la date de sortie
    release_date = game.find_all('td')[9].text.strip()
    release_dates.append(release_date)

# Création du dataframe pandas
df = pd.DataFrame({
    'Name': Name,
    'Publisher': publishers,
    'Global Sales': sales,
    'Date de sortie': release_dates
})

# Affichage du dataframe
df

,Name,Publisher,Global Sales,Date de sortie
0,Tetris,The Tetris Company,496.40m,01st Jan 88
1,Pokemon,Nintendo,464.84m,28th Sep 98
2,Call of Duty,Activision,400.00m,29th Oct 03
3,Super Mario,Nintendo,398.51m,20th Jul 83
4,Grand Theft Auto,Rockstar Games,395.00m,27th Mar 98
...,...,...,...,...
195,WarCraft,Blizzard Entertainment,19.00m,23rd Nov 94
196,Mario Kart 7 Read the review,Nintendo,18.97m,04th Dec 11
197,Super Mario Party Read the review,Nintendo,18.79m,05th Oct 18
198,Midnight Club,Rockstar Games,18.50m,26th Oct 00


In [ ]:
#test pour les Name que nous navons pas réussi à scrapper la première fois
Name = []

td_elements = soup.find_all('td', style='font-size:12pt;')

# parcourir chaque élément <td> et extraire le texte
for td in td_elements:
    titre = td.text.strip()
    Name.append(titre)

df_Name = pd.DataFrame({
    'Name': Name,
})    

df_Name

,Name
0,Tetris
1,Pokemon
2,Call of Duty
3,Super Mario
4,Grand Theft Auto
...,...
195,WarCraft
196,Mario Kart 7 Read the review
197,Super Mario Party Read the review
198,Midnight Club


cela fonctionne, on intègre cela au code final

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

page = 1
Name = []
publishers = []
sales = []
release_dates = []

while page != 100:
    url = f'https://www.vgchartz.com/games/games.php?page={page}&results=1000&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&month=&year=&sales_type=&headerBackground=&footerBackground='
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    td_elements = soup.find_all('td', style='font-size:12pt;')
    for td in td_elements:
        titre = td.text.strip()
        Name.append(titre)

    for game in soup.find_all('tr', {'style': ['background-image:url(../imgs/chartBar_large.gif); height:70px', 'background-image:url(../imgs/chartBar_alt_large.gif); height:70px', 'font-size:12pt;']}):
        publisher = game.find_all('td')[4].text.strip()
        publishers.append(publisher)

        sale = game.find_all('td')[8].text.strip()
        sales.append(sale)

        release_date = game.find_all('td')[9].text.strip()
        release_dates.append(release_date)

    page += 1

df = pd.DataFrame({
    'Name': Name,
    'Publisher': publishers,
    'Global Sales': sales,
    'Date de sortie': release_dates
})

df


,Name,Publisher,Global Sales,Date de sortie
0,Tetris,The Tetris Company,496.40m,01st Jan 88
1,Pokemon,Nintendo,464.84m,28th Sep 98
2,Call of Duty,Activision,400.00m,29th Oct 03
3,Super Mario,Nintendo,398.51m,20th Jul 83
4,Grand Theft Auto,Rockstar Games,395.00m,27th Mar 98
...,...,...,...,...
62689,Zombieland: Double Tap - Road Trip,GameMill Entertainment,N/A,15th Oct 19
62690,Zombillie,Forever Entertainment S.A.,N/A,29th Mar 18
62691,Zone of the Enders: The 2nd Runner MARS,Konami,N/A,06th Sep 18
62692,Zoo Tycoon: Ultimate Animal Collection,Microsoft Studios,N/A,31st Oct 17


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62694 entries, 0 to 62693
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name            62694 non-null  object
 1   Publisher       62694 non-null  object
 2   Global Sales    62694 non-null  object
 3   Date de sortie  62694 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [ ]:
#df.to_csv('VgChartz.csv')

Cleaning du DataFrame

In [ ]:
(df['Global Sales'] == 'N/A').value_counts()

#Beaucoup de NA dans la colonne Global Sales, tous les jeux n'ont pas d'informations concernant cette donnée.

True     58468
False     4226
Name: Global Sales, dtype: int64

In [ ]:
df['Publisher'].nunique()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt


In [ ]:
df = pd.read_csv('VgChartz.csv')
df.head(166)

,Unnamed: 0,Name,Publisher,Global Sales,Date de sortie
0,0,Tetris,The Tetris Company,496.40m,01st Jan 88
1,1,Pokemon,Nintendo,464.84m,28th Sep 98
2,2,Call of Duty,Activision,400.00m,29th Oct 03
3,3,Super Mario,Nintendo,398.51m,20th Jul 83
4,4,Grand Theft Auto,Rockstar Games,395.00m,27th Mar 98
...,...,...,...,...,...
161,161,Pokémon Scarlet / Violet Read the review,Nintendo,20.61m,18th Nov 22
162,162,Super Mario World,Nintendo,20.61m,13th Aug 91
163,163,Grand Theft Auto V Read the review,Rockstar Games,NaN,17th Sep 13
164,164,The Lord of the Rings,Melbourne House,20.17m,01st Nov 82


In [ ]:
df.drop('Unnamed: 0', axis=1)

,Name,Publisher,Global Sales,Date de sortie
0,Tetris,The Tetris Company,496.40m,01st Jan 88
1,Pokemon,Nintendo,464.84m,28th Sep 98
2,Call of Duty,Activision,400.00m,29th Oct 03
3,Super Mario,Nintendo,398.51m,20th Jul 83
4,Grand Theft Auto,Rockstar Games,395.00m,27th Mar 98
...,...,...,...,...
62689,Zombieland: Double Tap - Road Trip,GameMill Entertainment,NaN,15th Oct 19
62690,Zombillie,Forever Entertainment S.A.,NaN,29th Mar 18
62691,Zone of the Enders: The 2nd Runner MARS,Konami,NaN,06th Sep 18
62692,Zoo Tycoon: Ultimate Animal Collection,Microsoft Studios,NaN,31st Oct 17


In [ ]:
df.drop('Unnamed: 0', axis=1)

,Name,Publisher,Global Sales,Date de sortie
0,Tetris,The Tetris Company,496.40m,01st Jan 88
1,Pokemon,Nintendo,464.84m,28th Sep 98
2,Call of Duty,Activision,400.00m,29th Oct 03
3,Super Mario,Nintendo,398.51m,20th Jul 83
4,Grand Theft Auto,Rockstar Games,395.00m,27th Mar 98
...,...,...,...,...
62689,Zombieland: Double Tap - Road Trip,GameMill Entertainment,NaN,15th Oct 19
62690,Zombillie,Forever Entertainment S.A.,NaN,29th Mar 18
62691,Zone of the Enders: The 2nd Runner MARS,Konami,NaN,06th Sep 18
62692,Zoo Tycoon: Ultimate Animal Collection,Microsoft Studios,NaN,31st Oct 17


In [ ]:
df['Global Sales'] = df['Global Sales'].str.replace('m','')

#on retire le "m" de global sales pour n'avoir que des float, on pourra ensuite convertir en float.

In [ ]:
df=df.drop('Unnamed: 0', axis=1)

In [ ]:
df

,Name,Publisher,Global Sales,Date de sortie
0,Tetris,The Tetris Company,496.40,01st Jan 88
1,Pokemon,Nintendo,464.84,28th Sep 98
2,Call of Duty,Activision,400.00,29th Oct 03
3,Super Mario,Nintendo,398.51,20th Jul 83
4,Grand Theft Auto,Rockstar Games,395.00,27th Mar 98
...,...,...,...,...
62689,Zombieland: Double Tap - Road Trip,GameMill Entertainment,NaN,15th Oct 19
62690,Zombillie,Forever Entertainment S.A.,NaN,29th Mar 18
62691,Zone of the Enders: The 2nd Runner MARS,Konami,NaN,06th Sep 18
62692,Zoo Tycoon: Ultimate Animal Collection,Microsoft Studios,NaN,31st Oct 17


In [ ]:
df['Global Sales'] = df['Global Sales'].astype(float)
#conversion en float

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62694 entries, 0 to 62693
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            62694 non-null  object 
 1   Publisher       62694 non-null  object 
 2   Global Sales    4226 non-null   float64
 3   Date de sortie  56822 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.9+ MB


In [ ]:
df['Date de sortie'] = pd.to_datetime(df['Date de sortie'])
df['Année de sortie'] = df['Date de sortie'].dt.year
#conversion en format year



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62694 entries, 0 to 62693
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Name             62694 non-null  object        
 1   Publisher        62694 non-null  object        
 2   Global Sales     4226 non-null   float64       
 3   Date de sortie   56822 non-null  datetime64[ns]
 4   Année de sortie  56822 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 2.4+ MB


In [ ]:
df = df.drop("Date de sortie",axis=1)


In [ ]:
df

,Name,Publisher,Global Sales,Année de sortie
0,Tetris,The Tetris Company,496.40,1988.0
1,Pokemon,Nintendo,464.84,1998.0
2,Call of Duty,Activision,400.00,2003.0
3,Super Mario,Nintendo,398.51,1983.0
4,Grand Theft Auto,Rockstar Games,395.00,1998.0
...,...,...,...,...
62689,Zombieland: Double Tap - Road Trip,GameMill Entertainment,NaN,2019.0
62690,Zombillie,Forever Entertainment S.A.,NaN,2018.0
62691,Zone of the Enders: The 2nd Runner MARS,Konami,NaN,2018.0
62692,Zoo Tycoon: Ultimate Animal Collection,Microsoft Studios,NaN,2017.0
